<div style="text-align:center;font-size:22pt; font-weight:bold;color:white;border:solid black 1.5pt;background-color:#1e7263;">
    TensorBoard Callback Overview
</div>

In [1]:
# ======================================================================= #
# Course: Deep Learning Complete Course (CS-501)
# Author: Dr. Saad Laouadi
# Institution: Quant Coding Versity Academy
#
# ==========================================================
# Lesson: Understanding tensorboard callback
#         Synthetic Data Example
# ==========================================================
# ## Learning Objectives
# This example will enable you to:
# 1. Understand the tensorboard callback
# 2. Setup the environment for using tensorboard
# =======================================================================
#          Copyright © Dr. Saad Laouadi 2025
# =======================================================================

In [ ]:
# ============================================================================ #
#                         Environment Path Configuration                       #
# ============================================================================ #
#
# Purpose:
#   Configure the system PATH to use Python executables from the active virtual 
#   environment instead of global installations.
#
# Usage:
#   1. First verify if configuration is needed by running: !which python
#   2. If the output shows the global Python installation rather than your 
#      virtual environment, execute this configuration block
#
# Note:
#   This configuration is typically only needed for JupyterLab Desktop or 
#   similar standalone installations. Web-based JupyterLab or properly 
#   configured environments should not require this adjustment.
# ============================================================================ #

import os
import sys

env_path = os.path.dirname(sys.executable)
os.environ['PATH'] = f"{env_path}:{os.environ['PATH']}"

In [1]:
# ==================================================== #
#        Load Required Libraries
# ==================================================== #

import shutil
from datetime import datetime
import io


# Disable Metal API Validation
os.environ["METAL_DEVICE_WRAPPER_TYPE"] = "0"  

# import tensorflow
import tensorflow as tf

print("="*72)

%reload_ext watermark
%watermark -a "Dr. Saad Laouadi" -u -d -m

print("="*72)
print("Imported Packages and Their Versions:")
print("="*72)

%watermark -iv
print("="*72)

# Global Config
RANDOM_STATE = 101

Author: Dr. Saad Laouadi

Last updated: 2024-12-31

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 24.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Imported Packages and Their Versions:
sklearn   : 1.5.1
keras     : 3.6.0
matplotlib: 3.9.2
tensorflow: 2.16.2
seaborn   : 0.13.2
numpy     : 1.26.4
pandas    : 2.2.2



# Modern Hyperparameter Tuning with Keras: A Comprehensive Guide

## Introduction

The `tensorflow.keras.wrappers.scikit_learn` module, which previously provided the `KerasClassifier` and `KerasRegressor` wrappers, has been deprecated. This guide presents modern approaches to hyperparameter tuning for Keras models, offering both alternative solutions and best practices.

## Modern Alternatives

### 1. Using Keras Tuner

Keras Tuner is the official hyperparameter tuning library for Keras, offering a more powerful and flexible approach than the old scikit-learn wrappers.

```python
import keras_tuner as kt
from tensorflow import keras

def build_model(hp):
    model = keras.Sequential()
    # Tune number of units in the first dense layer
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    
    # Tune dropout rate
    hp_dropout = hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)
    model.add(keras.layers.Dropout(hp_dropout))
    
    # Tune learning rate
    hp_learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    return model

# Initialize tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=30,
    factor=3,
    directory='keras_tuner',
    project_name='hyperparameter_optimization'
)

# Perform hyperparameter search
tuner.search(x_train, y_train,
            epochs=30,
            validation_data=(x_val, y_val),
            callbacks=[keras.callbacks.EarlyStopping(patience=5)])

# Get best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
```

### 2. Custom Scikit-learn Compatible Wrapper

If you still need scikit-learn compatibility, you can create a custom wrapper:

```python
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np

class KerasClassifierWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, model_fn, **kwargs):
        self.model_fn = model_fn
        self.kwargs = kwargs
        self.model = None
        
    def fit(self, X, y, **kwargs):
        if self.model is None:
            self.model = self.model_fn(**self.kwargs)
        
        # Convert y to one-hot if needed
        if len(y.shape) == 1:
            y = keras.utils.to_categorical(y)
            
        self.model.fit(X, y, **kwargs)
        return self
        
    def predict(self, X):
        predictions = self.model.predict(X)
        return np.argmax(predictions, axis=1)
        
    def predict_proba(self, X):
        return self.model.predict(X)

# Usage example:
def create_model(units=64, dropout=0.2, learning_rate=0.001):
    model = keras.Sequential([
        keras.layers.Dense(units, activation='relu'),
        keras.layers.Dropout(dropout),
        keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Create wrapped classifier
classifier = KerasClassifierWrapper(
    model_fn=create_model,
    units=64,
    dropout=0.2,
    learning_rate=0.001
)
```

### 3. Integration with Scikit-learn's GridSearchCV

Using the custom wrapper with GridSearchCV:

```python
from sklearn.model_selection import GridSearchCV

param_grid = {
    'units': [32, 64, 128],
    'dropout': [0.1, 0.2, 0.3],
    'learning_rate': [0.001, 0.01]
}

grid_search = GridSearchCV(
    estimator=KerasClassifierWrapper(create_model),
    param_grid=param_grid,
    cv=3,
    scoring='accuracy',
    n_jobs=1  # Use 1 for GPU training
)

grid_search.fit(X_train, y_train)
```

## Best Practices for Hyperparameter Tuning

### 1. Search Space Design

When defining hyperparameter search spaces, consider these guidelines:

- Learning rate: Use log-uniform distribution (e.g., 1e-4 to 1e-2)
- Number of units: Use uniform distribution with exponential steps (32, 64, 128, etc.)
- Dropout rate: Linear space between 0.0 and 0.5
- Batch size: Powers of 2 (32, 64, 128, etc.)

### 2. Cross-Validation Strategy

```python
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
```

### 3. Resource Management

- Use early stopping to prevent overfitting and save computational resources
- Implement learning rate scheduling
- Monitor GPU memory usage

```python
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3,
        min_lr=1e-6
    )
]
```

## Advanced Techniques

### 1. Bayesian Optimization

Using `scikit-optimize` for Bayesian optimization:

```python
from skopt import BayesSearchCV
from skopt.space import Real, Integer

search_spaces = {
    'learning_rate': Real(1e-4, 1e-2, prior='log-uniform'),
    'units': Integer(32, 512),
    'dropout': Real(0.0, 0.5)
}

bayes_search = BayesSearchCV(
    KerasClassifierWrapper(create_model),
    search_spaces,
    n_iter=50,
    cv=3,
    scoring='accuracy'
)
```

### 2. Population-Based Training

For advanced users, population-based training combines hyperparameter optimization with neural architecture search:

```python
import ray
from ray import tune

def train_func(config):
    model = create_model(**config)
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=30,
        callbacks=[keras.callbacks.EarlyStopping(patience=5)]
    )
    tune.report(accuracy=max(history.history['val_accuracy']))

analysis = tune.run(
    train_func,
    config={
        "units": tune.grid_search([32, 64, 128]),
        "dropout": tune.uniform(0.0, 0.5),
        "learning_rate": tune.loguniform(1e-4, 1e-2)
    },
    num_samples=50
)
```

## Conclusion

While the deprecation of `keras.wrappers.scikit_learn` may initially seem challenging, modern alternatives offer more powerful and flexible approaches to hyperparameter tuning. The combination of Keras Tuner, custom wrappers, and advanced optimization techniques provides a robust framework for developing and optimizing deep learning models.

Remember to:
1. Choose the appropriate tuning strategy based on your computational resources
2. Design meaningful search spaces for your hyperparameters
3. Implement proper cross-validation and early stopping
4. Monitor and manage computational resources effectively

By following these guidelines and leveraging modern tools, you can achieve better results in hyperparameter optimization for your Keras models.